# Importing required Libraries and Modules

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.ion()

import torch
from torch.utils.data.dataset import random_split
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

import torchvision
from torchvision import models
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
torch.manual_seed(42)

import time
import copy
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Setting paths to training images

In [2]:
TRAIN_FOLDER_PATH = "/kaggle/input/intel-image-classification/seg_train/seg_train"

# Finding Mean and SD for Normalization

In [3]:
%%time

train = ImageFolder(
            TRAIN_FOLDER_PATH,
            transform = tt.Compose([
                tt.Resize(64),
                tt.RandomCrop(64),
                tt.ToTensor()
            ])
        )

train_dl_for_mean_sd = DataLoader(
                            dataset = train,
                            batch_size = 108,
                            shuffle = True,
                            num_workers = 2,
                            pin_memory = True
                        )

sum, ss, batches = 0, 0, 0
for data, _ in train_dl_for_mean_sd :
    sum += torch.mean(data, dim = ([0,2,3]))
    ss += torch.mean(data**2, dim =([0,2,3]))
    batches += 1
mean = sum / batches
std = ((ss / batches) - mean**2)**0.5
print( mean, std )

# Transforming the Images

In [3]:
mean = (0.4302, 0.4575, 0.4539)
std = (0.2483, 0.2467, 0.2806)
mean_sd = ( mean, std )

train_val_transform = tt.Compose([
                            tt.Resize(64),
                            tt.RandomCrop(64),
                            tt.RandomHorizontalFlip(),
                            tt.ToTensor(),
                            tt.Normalize(*mean_sd, inplace = True)
                        ])

train = ImageFolder(TRAIN_FOLDER_PATH, train_val_transform)

# Train - Validation Split

In [4]:
VAL_RATIO = 0.2
VAL_SIZE = int(len(train)*VAL_RATIO)
TRAIN_SIZE = len(train) - VAL_SIZE
train_data, val_data = random_split(train, [TRAIN_SIZE, VAL_SIZE])

# DataLoaders

In [5]:
BATCH_SIZE = 72

train_data_loader = DataLoader(
                            dataset = train_data,
                            batch_size = BATCH_SIZE,
                            shuffle = True,
                            num_workers = 2,
                            pin_memory = True
                        )

valid_data_loader = DataLoader(
                            dataset = val_data,
                            batch_size = BATCH_SIZE,
                            shuffle = True,
                            num_workers = 2,
                            pin_memory = True
                    )

# Making Dictionaries

In [6]:
image_datasets = {
                    "train" : train_data,
                    "val"   : val_data
                 }

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = train.classes

dataloaders = {
                "train" : train_data_loader,
                "val" : valid_data_loader
              }

# Batch Image Visualization

In [7]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.4291, 0.4563, 0.4526])
    std = np.array([0.2485, 0.2472, 0.2810])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out)

# Configuration of device

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Defining functions for training and visualizing

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, -1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    model.load_state_dict(best_model_wts)
    return model

In [10]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

# Using ResNet18

In [11]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Training

In [12]:
model_ft = train_model(
                model_ft, 
                criterion, 
                optimizer_ft,
                exp_lr_scheduler,
                num_epochs = 25
            )

# Output Visualizations

In [14]:
visualize_model(model_ft)